<a href="https://colab.research.google.com/github/sabamisoon/AutoFX/blob/master/AutoFX_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install oandapyV20

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for oandapyV20: filename=oandapyV20-0.6.3-cp36-none-any.whl size=69742 sha256=bd321a7feb322ce175bfc56e1a6e551c33486b547ea4527671ebf3c5b51ee7cc
  Stored in directory: /root/.cache/pip/wheels/2c/b0/cd/49054320a6cc51db0ff4f1ee15d8840375f16cef57b81ebfc0
Successfully built oandapyV20


In [0]:
from oandapyV20 import API
accountID = "101-009-15091813-001"
access_token = "aaf617fd201e300e07f827bd55f91f5d-12cc2f32f07655a2b4fddb47cbaf483f"
api = API(access_token=access_token ,environment="practice")

In [0]:
import json
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream

instruments = "EUR_USD,EUR_JPY"
s = PricingStream(accountID=accountID, params={"instruments":instruments})
MAXREC = 10
try:
    n = 0
    for R in api.request(s):
        print(json.dumps(R, indent=2))
        n += 1
        if n > MAXREC:
            s.terminate("maxrecs received {}".format(MAXREC)) 
except V20Error as e:
    print("Error: {}".format(e))

In [9]:
import json
import pandas as pd
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.types import DateTime
from datetime import datetime ,timedelta
from dateutil.relativedelta import relativedelta

def get_equity_df( since ,until ,interval ,price ) :
    cnt = 'mid' if price=='M' else 'ask' if price=='A' else 'bid' if price=='B' else 'mid'
    params = { "from": DateTime( since ).value
              ,"to": DateTime( until ).value
              ,"price": price
              ,"granularity": interval }
    r = instruments.InstrumentsCandles( instrument="USD_JPY" ,params=params )
    api.request( r )    
    raw_list = []
    for raw in r.response['candles'] :
        raw_list.append( [ raw['time'] ,raw[cnt]['o'] ,raw[cnt]['h'] ,raw[cnt]['l'] ,raw[cnt]['c'] ,raw['volume'] ] )
    raw_df = pd.DataFrame( raw_list ,columns=[ 'Time' ,f'Open_{cnt}' ,f'High_{cnt}' ,f'Low_{cnt}' ,f'Close_{cnt}' ,'Volume' ] )
    return raw_df


start    = datetime.strptime( '2005-01-01' ,'%Y-%m-%d' )
end      = datetime.strptime( '2019-12-31' ,'%Y-%m-%d' )
month    = 12 * 2 #上限5000レコード
interval = 'H4'
restart  = 0

since = start + relativedelta( months=( restart ) )
until = start + relativedelta( months=( month + restart ) )
df = pd.DataFrame()

while True :
    if until > datetime.now() :
        until = datetime.now()
        if relativedelta( since ,until ).months==0 : break
    
    print( f'since:{ since } ,until:{ until }' )

    raw_a = get_equity_df( since ,until ,interval ,'A' )
    raw_b = get_equity_df( since ,until ,interval ,'B' )
    raw = pd.merge( raw_a ,raw_b )
    raw = raw.set_index( 'Time' )
    raw.index = pd.to_datetime( raw.index )

    df = pd.concat( [ df ,raw ] )

    since = until
    until = until + relativedelta( months=month )
    if since >= end : break

since:2005-01-01 00:00:00 ,until:2007-01-01 00:00:00
since:2007-01-01 00:00:00 ,until:2009-01-01 00:00:00
since:2009-01-01 00:00:00 ,until:2011-01-01 00:00:00
since:2011-01-01 00:00:00 ,until:2013-01-01 00:00:00
since:2013-01-01 00:00:00 ,until:2015-01-01 00:00:00
since:2015-01-01 00:00:00 ,until:2017-01-01 00:00:00
since:2017-01-01 00:00:00 ,until:2019-01-01 00:00:00
since:2019-01-01 00:00:00 ,until:2020-06-03 14:23:14.454150


In [10]:
import oandapyV20.endpoints.orders as orders
order_data = {
        "order": {
          "instrument": "USD_JPY",
            "units": "+100",
            "type": "MARKET",
          }
      }
o = orders.OrderCreate(accountID, data=order_data)
api.request( o )
o.response

{'lastTransactionID': '5',
 'orderCreateTransaction': {'accountID': '101-009-15091813-001',
  'batchID': '4',
  'id': '4',
  'instrument': 'USD_JPY',
  'positionFill': 'DEFAULT',
  'reason': 'CLIENT_ORDER',
  'requestID': '60717148028986081',
  'time': '2020-06-03T14:24:32.733444035Z',
  'timeInForce': 'FOK',
  'type': 'MARKET_ORDER',
  'units': '100',
  'userID': 15091813},
 'orderFillTransaction': {'accountBalance': '3000000.0000',
  'accountID': '101-009-15091813-001',
  'batchID': '4',
  'commission': '0.0000',
  'financing': '0.0000',
  'fullPrice': {'asks': [{'liquidity': '250000', 'price': '108.736'}],
   'bids': [{'liquidity': '250000', 'price': '108.732'}],
   'closeoutAsk': '108.741',
   'closeoutBid': '108.727',
   'timestamp': '2020-06-03T14:24:31.497655798Z'},
  'fullVWAP': '108.736',
  'gainQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '0.2000',
  'id': '5',
  'instrument': 'USD_JPY',
  'lossQuoteHomeConversionFactor': '1',
  '

In [11]:
import oandapyV20.endpoints.orders as orders
order_data = {
        "order": {
           "price": "107.765"
          ,"instrument": "USD_JPY"
          ,"units": "+10000"
          ,"type": "STOP"
          }
      }
o = orders.OrderCreate(accountID, data=order_data)
api.request( o )
o.response

{'lastTransactionID': '7',
 'orderCreateTransaction': {'accountID': '101-009-15091813-001',
  'batchID': '6',
  'id': '6',
  'instrument': 'USD_JPY',
  'partialFill': 'DEFAULT',
  'positionFill': 'DEFAULT',
  'price': '107.765',
  'reason': 'CLIENT_ORDER',
  'requestID': '42702749883017247',
  'time': '2020-06-03T14:25:58.142172483Z',
  'timeInForce': 'GTC',
  'triggerCondition': 'DEFAULT',
  'type': 'STOP_ORDER',
  'units': '10000',
  'userID': 15091813},
 'orderFillTransaction': {'accountBalance': '3000000.0000',
  'accountID': '101-009-15091813-001',
  'batchID': '6',
  'commission': '0.0000',
  'financing': '0.0000',
  'fullPrice': {'asks': [{'liquidity': '250000', 'price': '108.706'}],
   'bids': [{'liquidity': '250000', 'price': '108.702'}],
   'closeoutAsk': '108.711',
   'closeoutBid': '108.698',
   'timestamp': '2020-06-03T14:25:57.987001477Z'},
  'fullVWAP': '108.706',
  'gainQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '20.0000',

In [12]:
import oandapyV20.endpoints.orders as orders
c = orders.OrdersPending( accountID )
api.request( c )
c.response

{'lastTransactionID': '7', 'orders': []}

In [13]:
import oandapyV20.endpoints.orders as orders
c = orders.OrderDetails( accountID ,orderID='4' )
api.request( c )
c.response

{'lastTransactionID': '7',
 'order': {'createTime': '2020-06-03T14:24:32.733444035Z',
  'filledTime': '2020-06-03T14:24:32.733444035Z',
  'fillingTransactionID': '5',
  'id': '4',
  'instrument': 'USD_JPY',
  'positionFill': 'DEFAULT',
  'state': 'FILLED',
  'timeInForce': 'FOK',
  'tradeOpenedID': '5',
  'type': 'MARKET',
  'units': '100'}}

In [14]:
import oandapyV20.endpoints.positions as positions
position_data = { "longUnits": "ALL" }
p = positions.PositionClose( accountID=accountID, data=position_data, instrument="USD_JPY" )
api.request( p )
p.response

{'lastTransactionID': '9',
 'longOrderCreateTransaction': {'accountID': '101-009-15091813-001',
  'batchID': '8',
  'id': '8',
  'instrument': 'USD_JPY',
  'longPositionCloseout': {'instrument': 'USD_JPY', 'units': 'ALL'},
  'positionFill': 'REDUCE_ONLY',
  'reason': 'POSITION_CLOSEOUT',
  'requestID': '78731547524149902',
  'time': '2020-06-03T14:28:27.179072532Z',
  'timeInForce': 'FOK',
  'type': 'MARKET_ORDER',
  'units': '-10100',
  'userID': 15091813},
 'longOrderFillTransaction': {'accountBalance': '2999815.2000',
  'accountID': '101-009-15091813-001',
  'batchID': '8',
  'commission': '0.0000',
  'financing': '0.0000',
  'fullPrice': {'asks': [{'liquidity': '250000', 'price': '108.692'}],
   'bids': [{'liquidity': '250000', 'price': '108.688'}],
   'closeoutAsk': '108.697',
   'closeoutBid': '108.684',
   'timestamp': '2020-06-03T14:28:27.014828473Z'},
  'fullVWAP': '108.688',
  'gainQuoteHomeConversionFactor': '1',
  'guaranteedExecutionFee': '0.0000',
  'halfSpreadCost': '20.

In [15]:
import oandapyV20.endpoints.positions as positions
p = positions.PositionDetails(accountID=accountID, instrument="USD_JPY")
api.request( p )
p.respons

AttributeError: ignored